In [1]:
import numpy as np
from keras.models import load_model
from sklearn.model_selection import KFold
from time import time
from timeit import default_timer as timer
import pickle


Using TensorFlow backend.


In [2]:
dataset = np.load('all_dataset_window19Middle.npy') # load

In [3]:
windowSize = 19
predictionIndex = 9
classSize = 8  # 2 or 3 
numberOfFeatures = 50 #44 

amino_acid_residues = 21
num_classes = 8

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv1D, AveragePooling1D, MaxPooling1D, TimeDistributed, LeakyReLU, BatchNormalization, Flatten
from keras import optimizers, callbacks
from keras import regularizers
from keras.utils import plot_model
import os

LR = 0.0005
drop_out = 0.3
batch_dim = 64
nn_epochs = 10
w_reg = regularizers.l2(0.0001)
number_filters = 16


def get_model():
   

    loss = 'categorical_crossentropy'


    m = Sequential()

#first convolutional neural netwok
    m.add(Conv1D( 16 , 3,  strides=1, padding='same', activation='relu', use_bias=True, input_shape=(windowSize, 21), kernel_regularizer=w_reg))
    m.add(BatchNormalization())

    m.add(Conv1D( 16, 3,  strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg))
    m.add(BatchNormalization())
    m.add(Conv1D( 16, 5,  strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg))
    m.add(BatchNormalization())
    m.add(Conv1D( 16, 7,  strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg))
    m.add(BatchNormalization())


    m.add(Conv1D( 16, 3,  strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg))
    m.add(BatchNormalization())
    m.add(Conv1D( 16, 5,  strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg))
    m.add(BatchNormalization())
    m.add(Conv1D( 16, 7,  strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg))
    m.add(BatchNormalization())

    m.add(Conv1D( 16, 3,  strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg))
    m.add(BatchNormalization())
    m.add(Conv1D( 16, 5,  strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg))
    m.add(BatchNormalization())
    m.add(Conv1D( 16, 7,  strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg))
    m.add(BatchNormalization())



    #4 dense layer
    m.add(Dense(200, activation='relu', use_bias=True,  kernel_regularizer=w_reg))

    #5 softmax output layer
    m.add(Dense(num_classes, activation = 'softmax'))

    opt = optimizers.Adam(lr=LR)
    m.compile(optimizer=opt, loss=loss,metrics=['accuracy', 'mae'])

    print("\nHyper Parameters\n")
    print("Learning Rate: " + str(LR))
    print("Drop out: " + str(drop_out))

    print("Batch dim: " + str(batch_dim))
    print("Number of epochs: " + str(nn_epochs))
    print("Regularizers: " + str(w_reg.l2))
    print("\nLoss: " + loss + "\n")
    m.summary()

    #os.environ["PATH"] += os.pathsep + 'C:/Users/Ieremie/Anaconda3/pkgs/graphviz-2.38-hfd603c8_2/Library/bin'
    #plot_model(m)
    
    return m
    

In [5]:
print(dataset.shape)
print(dataset[0,:3,:])

(1133502, 19, 50)
[[0.         0.         0.         0.         0.         1.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         1.         0.         0.
  0.         0.         0.         0.         0.         0.02145729
  0.01365366 0.0054863  0.01212843 0.99848372 0.0066267  0.2748805
  0.0765622  0.01448573 0.11815697 0.09708863 0.01261716 0.00570892
  0.01852383 0.01763634 0.02104135 0.02484354 0.05625294 0.51499552
  0.26894143 0.99938297]
 [0.         0.         0.         1.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         1.         0.         0.
  0.         0.         0.         0.         0.         0.05732417
  0.00669285 0.9976241  0.96832377 0.00682713 0.14185107 0.09885607
  0.00919371 0.80059224 0.00784502 0.01763634 0.55971366 0.0541667
  0

In [7]:
from sklearn.utils import shuffle
dataset = shuffle(dataset, random_state=111)
print(dataset.shape)
print(dataset[0,:3,:])

(1133502, 19, 50)
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 9.79567766e-01 9.98887479e-01 4.71085456e-04
  8.66657007e-04 2.65143369e-03 2.44807987e-03 6.10879273e-04
  9.74667370e-01 1.16951030e-03 8.58148873e-01 8.16078246e-01
  6.88710774e-04 9.77045507e-04 4.40763310e-03 7.76451314e-04
  1.20256864e-01 1.14302030e-02 6.27147734e-01 6.55144104e-04
  2.68941432e-01 1.42807304e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.

In [6]:
kf = KFold(n_splits=10, random_state=10, shuffle=True)
for train, test in kf.split(dataset):
    print(train,test)

[      1       2       4 ... 1133498 1133499 1133500] [      0       3       6 ... 1133480 1133485 1133501]
[      0       1       2 ... 1133499 1133500 1133501] [      5       7       8 ... 1133465 1133487 1133491]
[      0       1       2 ... 1133498 1133500 1133501] [     10      21      29 ... 1133495 1133496 1133499]
[      0       1       2 ... 1133499 1133500 1133501] [     11      20      22 ... 1133478 1133490 1133497]
[      0       1       2 ... 1133499 1133500 1133501] [      4      35      36 ... 1133481 1133482 1133494]
[      0       1       2 ... 1133499 1133500 1133501] [      9      42      44 ... 1133473 1133488 1133492]
[      0       2       3 ... 1133499 1133500 1133501] [      1      25      27 ... 1133461 1133466 1133469]
[      0       1       2 ... 1133499 1133500 1133501] [     12      16      17 ... 1133460 1133486 1133493]
[      0       1       2 ... 1133499 1133500 1133501] [     19      23      28 ... 1133470 1133483 1133498]
[      0       1       3 ...

In [8]:
l = 0
r = 113350
n = 1133502

fold = 0
valid_indexes = [()]

#model 2
r += 113350
l += 113350
fold += 1

#model 3
r += 113350
l += 113350
fold += 1

for i in range (10):
  
    fold += 1
    print(fold)
    
    m = get_model()
    start_time = timer()
    history = m.fit(np.concatenate((dataset[0:l], dataset[r:n]), axis=0)[:,:,0:21], np.concatenate((dataset[0:l], dataset[r:n]), axis=0)[:,:,21 : (21+classSize)],
                    epochs=nn_epochs, validation_data=(dataset[l:r][:,:,0:21], dataset[l:r][:,:,21 : (21+classSize)]) ,shuffle=True,batch_size=batch_dim)
    end_time = timer()
    print("\n\nTime elapsed: " + "{0:.2f}".format((end_time - start_time)) + " s")
    
    scores = m.evaluate(dataset[l:r][:,:,0:21], dataset[l:r][:,:,21 : (21+classSize)])
    print("Loss: " + str(scores[0]) + ", Accuracy: " + str(scores[1]) + ", MAE: " + str(scores[2]) + ' fold ' + str(fold))
    
    
    m.save('model_kfold' + str(fold) + '.h5')  # creates a HDF5 file 
    with open('model_scores_kfold' + str(fold), 'wb') as file_pi:
        pickle.dump(scores, file_pi)
    with open('model_history_kfold' + str(fold), 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
    
    valid_indexes.append((l, r))
    r += 113350
    l += 113350

with open('model_kfold_indexes', 'wb') as file_pi:
    pickle.dump(valid_indexes, file_pi)   

3

Hyper Parameters

Learning Rate: 0.0005
Drop out: 0.3
Batch dim: 64
Number of epochs: 10
Regularizers: 1e-04

Loss: categorical_crossentropy

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 19, 16)            1024      
_________________________________________________________________
batch_normalization_1 (Batch (None, 19, 16)            64        
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 19, 16)            784       
_________________________________________________________________
batch_normalization_2 (Batch (None, 19, 16)            64        
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 19, 16)            1296      
_________________________________________________________________
batch_normalization_3 (Batch (None, 19, 1

KeyboardInterrupt: 

In [14]:
kf = KFold(n_splits=10, random_state=10, shuffle=True)
fold = 0
valid_indexes = [([],[])]
for train, test in kf.split(dataset):

    fold += 1
    print(fold)
    
    m = get_model()
    start_time = timer()
    history = m.fit(dataset[train][:,:,0:21], dataset[train][:,:,21 : (21+classSize)], epochs=nn_epochs,
                    batch_size=batch_dim, validation_data=(dataset[test][:,:,0:21], dataset[test][:,:,21 : (21+classSize)]),shuffle=True)
    end_time = timer()
    print("\n\nTime elapsed: " + "{0:.2f}".format((end_time - start_time)) + " s")

    
    scores = m.evaluate(dataset[test][:,:,0:21], dataset[test][:,:,21 : (21+classSize)])
    print("Loss: " + str(scores[0]) + ", Accuracy: " + str(scores[1]) + ", MAE: " + str(scores[2]) + ' fold ' + str(fold))
    
    
    m.save('model_kfold' + str(fold) + '.h5')  # creates a HDF5 file 
    with open('model_scores_kfold' + str(fold), 'wb') as file_pi:
        pickle.dump(scores, file_pi)
    with open('model_history_kfold' + str(fold), 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
    
    valid_indexes.append((train, test))

with open('model_kfold_indexes', 'wb') as file_pi:
    pickle.dump(valid_indexes, file_pi)    
    

1

Hyper Parameters

Learning Rate: 0.0005
Drop out: 0.3
Batch dim: 64
Number of epochs: 10
Regularizers: 1e-04

Loss: categorical_crossentropy

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_21 (Conv1D)           (None, 19, 16)            6400      
_________________________________________________________________
batch_normalization_21 (Batc (None, 19, 16)            64        
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 19, 16)            784       
_________________________________________________________________
batch_normalization_22 (Batc (None, 19, 16)            64        
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 19, 16)            1296      
_________________________________________________________________
batch_normalization_23 (Batc (None, 19, 1

KeyboardInterrupt: 